In [1]:
test_data_path = '/mnt/WORK/Project.DataScience/ml-exp/nlp/app/data1.dat'

In [2]:
test_data_batch = []
with open(test_data_path, 'r', encoding='utf-8') as fp:
    one_batch = []
    for line in fp.readlines():
        line = line.strip()
        one_batch.append(line)
        if '' == line:
            test_data_batch.append(one_batch)
            one_batch = []
            continue
    if 0 != len(one_batch):
        test_data_batch.append(one_batch)
            

In [3]:
test_data_batch_size = [len(batch) for batch in test_data_batch]

In [4]:
print(test_data_batch_size)

[25, 13, 11, 16, 10, 18, 12, 15, 7, 17, 18, 12, 21, 21, 27, 30, 17, 51, 20, 23, 12, 30, 36, 12, 34, 17, 28, 30, 38, 18, 28, 12, 18, 26, 21, 10, 6, 45, 7, 16, 13, 21, 25, 11, 15, 21, 30, 28, 20, 20, 11, 6, 14, 13, 7, 9, 11, 17, 9, 11, 18, 27, 35, 23, 11, 39, 22, 7, 8, 7, 7, 10, 7, 7, 11, 22, 11, 14, 14, 23, 7, 14, 10, 11, 15, 12, 11, 20, 18, 8, 11, 23, 10]


In [5]:
# 二元打分
def score_binary(pos1, pos2):
    return 1 if pos1 < pos2 else 0

In [6]:
# 句子抽取
# 我们认为, 只有相邻的句子才有相关性
# 句子的取法为:
#   给定一个窗口长度
#   从中抽取句子对
#   

from functools import reduce
import random
import time
import itertools
    
random.seed(time.time())

def random_sample_key(m, w, n, k, unique=True):
    """
    随机从m个样本中取出k个排列, 每个排列大小是n, 要求不重复
    :param m: 
    :param n: 
    :return: 
    """
    # 首先, 全排列的个数为 m*(m-1)...*(m-n+1) 如果超过这个数, 是不可能不重复的
    if w < n:
        raise RuntimeError("样本序列长度不能超过窗口长度")
    
    # 首先, 每个窗口的全排列的总数是
    full_permutation_num_per_window = reduce(lambda x, y: x*y, range(w-n+1, w+1))
    
    # TODO 实际上真实的样本空间比这个大, 但先用小的进行计算
    total_sampling_space_size = (m//w) * full_permutation_num_per_window
    
    if k > total_sampling_space_size:
        return []
    
    # 先随机生产取样窗口, 
    
    windows_b = list(range(0, m-w)) 
    random.shuffle(windows_b)

    sample_set = set()
    # 随机地去窗口, 生成全排列, 加入到样本序列集里
    i = 0
    while i < len(windows_b):
        w_b = windows_b[i]
        w_e = w_b + w
        for s in itertools.permutations(range(w_b, w_e), n):
            sample_set.add(s)
            if len(sample_set) >= k:
                return list(sample_set)
        i += 1
    
    return list(sample_set)

In [7]:
print(random_sample_key(10, 5, 2, 32))


[(4, 7), (1, 3), (4, 8), (5, 6), (2, 1), (0, 3), (8, 5), (2, 4), (5, 8), (1, 2), (6, 7), (7, 6), (0, 4), (8, 6), (6, 4), (5, 4), (4, 5), (1, 4), (7, 5), (2, 3), (8, 7), (1, 0), (6, 5), (0, 1), (4, 6), (6, 8), (5, 7), (7, 4), (2, 0), (7, 8), (0, 2), (8, 4)]


In [8]:
def corpusToSample(corpus, batch_size, score_func):
    return [
        (corpus[a], corpus[b], score_func(a, b)) 
        for a, b in random_sample_key(len(corpus), 5, 2, batch_size)
    ]

In [9]:
for batch in test_data_batch:
    # print(corpusToSample(batch, 32, score_binary))
    pass

In [10]:
structed_test_data = [corpusToSample(batch, 32, score_binary) for batch in test_data_batch]

In [11]:
### 将数据拆分为6个输入

def slice(x: str):
    items = x.split("|")
    dep = items[0::3]
    op1 = items[1::3]
    op2 = items[2::3]
    # tokenizer
    # embedding

    return dep, op1, op2

In [12]:
max_len = 50

def process_sample(batch):
    """
    [(sentence1, sentence2, y)]
    :param batch: 
    :return: 
    """
    dep_a = []
    op1_a = []
    op2_a = []
    dep_b = []
    op1_b = []
    op2_b = []
    y = []
    for (sentence1, sentence2, _y) in batch:
        _dep_a, _op1_a, _op2_a = slice(sentence1)
        _dep_b, _op1_b, _op2_b = slice(sentence2)
        dep_a.extend(_dep_a)
        dep_b.extend(_dep_b)
        op1_a.extend(_op1_a)
        op1_b.extend(_op1_b)
        op2_a.extend(_op2_a)
        op2_b.extend(_op2_b)
        y.append(_y)

    return dep_a, op1_a, op2_a, dep_b, op1_b, op2_b, y
    
    

In [13]:
# 获取依存标签个数

dep_set = set()
vocab = set()

for batch in structed_test_data:
    dep_a, op1_a, op2_a, dep_b, op1_b, op2_b, y = process_sample(batch)
    dep_set.update(dep_a)
    dep_set.update(dep_b)
    vocab.update(op1_a)
    vocab.update(op1_b)
    vocab.update(op2_b)
    vocab.update(op2_a)
    




In [14]:
print(dep_set)
print(len(vocab))

{'WP', 'LAD', 'DBL', 'POB', 'VOB', 'SBV', 'ATT', 'ADV', 'RAD', 'FOB', 'COO', 'IOB', 'CMP'}
5055


In [15]:
from collections import defaultdict
dep_token = defaultdict(lambda :0)

dep_token.update(
    
    {
    'SBV': 1,   #	主谓关系	subject-verb	我送她一束花 (我 <-- 送)
    'VOB': 2,   #	动宾关系	直接宾语，verb-object	我送她一束花 (送 --> 花)
    'IOB': 3,   #	间宾关系	间接宾语，indirect-object	我送她一束花 (送 --> 她)
    'FOB': 4,   #	前置宾语	前置宾语，fronting-object	他什么书都读 (书 <-- 读)
    'DBL': 5,   #	兼语	double	他请我吃饭 (请 --> 我)
    'ATT': 6,   #	定中关系	attribute	红苹果 (红 <-- 苹果)
    'ADV': 7,   #	状中结构	adverbial	非常美丽 (非常 <-- 美丽)
    'CMP': 8,   #	动补结构	complement	做完了作业 (做 --> 完)
    'COO': 9,   #	并列关系	coordinate	大山和大海 (大山 --> 大海)
    'POB': 10,  #	介宾关系	preposition-object	在贸易区内 (在 --> 内)
    'LAD': 11,  #	左附加关系	left adjunct	大山和大海 (和 <-- 大海)
    'RAD': 12,  #	右附加关系	right adjunct	孩子们 (孩子 --> 们)
    'IS':  13,  #	独立结构	independent structure	两个单句在结构上彼此独立
    'WP':  14,  #	标点符号	punctuation	标点符号
    'HED': 15,  #	核心关系	head	指整个句子的核心
}
)

def dep_tokenizer(dep):
    return dep_token[dep]


In [16]:
print(dep_tokenizer('VOB'))

2


In [17]:
from nlp.embedding import SimpleEmbeddingBuilder
SimpleEmbeddingBuilder = SimpleEmbeddingBuilder.SimpleEmbeddingBuilder

In [19]:
embedding_builder = SimpleEmbeddingBuilder(
    '/mnt/WORK/Project.DataScience/data/glove/glove.840B.300d.txt')

embedding, tokenizer = embedding_builder.build(vocab, cache='embedding_test_model_6')

custom embedding cache not found


In [22]:
print(tokenizer('word'))

0
